<a href="https://colab.research.google.com/github/ovimasbul83/ovimasbul83/blob/main/flowersrecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ovihaider","key":"4b1047900f51dd329793ea188b962d15"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d alxmamaev/flowers-recognition

 99% 444M/450M [00:04<00:00, 89.1MB/s]
100% 450M/450M [00:04<00:00, 94.5MB/s]


In [ ]:
import zipfile
local_zip='/content/flowers-recognition.zip'
zipref=zipfile.ZipFile(local_zip,'r')
zipref.extractall('/tmp/Flowers')
zipref.close()

In [ ]:
import os
import random
from shutil import copyfile

In [ ]:
try:
  os.mkdir('/tmp/Flowers/flower')
  os.mkdir('/tmp/Flowers/flower/training')
  os.mkdir('/tmp/Flowers/flower/training/daisy')
  os.mkdir('/tmp/Flowers/flower/training/dandelion')
  os.mkdir('/tmp/Flowers/flower/training/rose')
  os.mkdir('/tmp/Flowers/flower/training/sunflower')
  os.mkdir('/tmp/Flowers/flower/training/tulip')
  os.mkdir('/tmp/Flowers/flower/test')
  os.mkdir('/tmp/Flowers/flower/test/daisy')
  os.mkdir('/tmp/Flowers/flower/test/dandelion')
  os.mkdir('/tmp/Flowers/flower/test/rose')
  os.mkdir('/tmp/Flowers/flower/test/sunflower')
  os.mkdir('/tmp/Flowers/flower/test/tulip')
except OSError:
  pass

In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if (os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file size! i.e Zero length.')
    
    train_data_length = int(len(dataset) * SPLIT_SIZE)
    test_data_length = int(len(dataset) - train_data_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = shuffled_set[0:train_data_length]
    test_set = shuffled_set[-test_data_length:]
    
    for unitData in train_set:
        temp_train_data = SOURCE + unitData
        final_train_data = TRAINING + unitData
        copyfile(temp_train_data, final_train_data)
    
    for unitData in test_set:
        temp_test_data = SOURCE + unitData
        final_test_data = TESTING + unitData
        copyfile(temp_train_data, final_test_data)

In [ ]:
daisysource='/tmp/Flowers/flowers/flowers/daisy/'
daisytrain='/tmp/Flowers/flower/training/daisy/'
daisytest='/tmp/Flowers/flower/test/daisy/'
dandelionsource='/tmp/Flowers/flowers/dandelion/'
dandeliontrain='/tmp/Flowers/flower/training/dandelion/'
dandeliontest='/tmp/Flowers/flower/test/dandelion/'
rosesource='/tmp/Flowers/flowers/rose/'
rosetrain='/tmp/Flowers/flower/training/rose/'
rosetest='/tmp/Flowers/flower/test/rose/'
sunflowersource='/tmp/Flowers/flowers/sunflower/'
sunflowertrain='/tmp/Flowers/flower/training/sunflower/'
sunflowertest='/tmp/Flowers/flower/test/sunflower/'
tulipsource='/tmp/Flowers/flowers/tulip/'
tuliptrain='/tmp/Flowers/flower/training/tulip/'
tuliptest='/tmp/Flowers/flower/test/tulip/'

In [ ]:
split_size=0.9
split_data(daisysource,daisytrain,daisytest,split_size)
split_data(dandelionsource,dandeliontrain,dandeliontest,split_size)
split_data(rosesource,rosetrain,rosetest,split_size)
split_data(sunflowersource,sunflowertrain,sunflowertest,split_size)
split_data(tulipsource,tuliptrain,tuliptest,split_size)

In [ ]:
print(len(os.listdir(daisytrain)))
print(len(os.listdir(daisytest)))
print(len(os.listdir(dandeliontrain)))
print(len(os.listdir(dandeliontest)))


692
77
949
106


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

In [ ]:

training_dir='/tmp/Flowers/flower/training'
test_dir='/tmp/Flowers/flower/test'
traindatagen=ImageDataGenerator(rescale=1/255,rotation_range=10,width_shift_range=0.2,height_shift_range=0.2,shear_range=45.0,zoom_range=0.2,fill_mode='nearest',horizontal_flip=True)
testdatagen=ImageDataGenerator(rescale=1/255)
traindata=traindatagen.flow_from_directory(training_dir,target_size=(256,256),class_mode='categorical',batch_size=5)
testdata=testdatagen.flow_from_directory(test_dir,target_size=(256,256),class_mode='categorical',batch_size=5)
traindata.class_indices

Found 3888 images belonging to 5 classes.
Found 435 images belonging to 5 classes.


In [ ]:
model=tf.keras.models.Sequential([tf.keras.layers.Conv2D(32,(5,5),padding='same',input_shape=(256,256,3),activation='relu'),
                                  tf.keras.layers.MaxPooling2D((2,2)),
                                  tf.keras.layers.Conv2D(64,(3,3),activation='relu',padding='same'),
                                  tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)),
                                  tf.keras.layers.Conv2D(96,(3,3),activation='relu',padding='same'),
                                  tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)),
                                  tf.keras.layers.Conv2D(96,(3,3),activation='relu',padding='same'),
                                  tf.keras.layers.MaxPooling2D((2,2),strides=(2,2)),
                                  tf.keras.layers.Flatten(),
                                  #tf.keras.layers.Dense(64,activation='relu'),
                                  tf.keras.layers.Dropout(0.2),
                                  tf.keras.layers.Dense(512,activation='relu'),
                                  tf.keras.layers.Dense(5,activation='softmax')])
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_81 (Conv2D)           (None, 256, 256, 32)      2432      
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 128, 128, 64)      18496     
_________________________________________________________________
max_pooling2d_82 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_83 (Conv2D)           (None, 64, 64, 96)        55392     
_________________________________________________________________
max_pooling2d_83 (MaxPooling (None, 32, 32, 96)        0         
_________________________________________________________________
conv2d_84 (Conv2D)           (None, 32, 32, 96)      

In [ ]:
history=model.fit(traindata,steps_per_epoch=2,epochs=50,verbose=2,validation_data=testdata)

Epoch 1/50
2/2 - 2s - loss: 1.7371 - accuracy: 0.1000 - val_loss: 1.6145 - val_accuracy: 0.2276
Epoch 2/50
2/2 - 2s - loss: 1.4555 - accuracy: 0.4000 - val_loss: 1.7351 - val_accuracy: 0.2437
Epoch 3/50
2/2 - 2s - loss: 2.0128 - accuracy: 0.0000e+00 - val_loss: 1.7197 - val_accuracy: 0.2437
Epoch 4/50
2/2 - 2s - loss: 1.4701 - accuracy: 0.4000 - val_loss: 1.7027 - val_accuracy: 0.2437
Epoch 5/50
2/2 - 2s - loss: 1.8522 - accuracy: 0.1000 - val_loss: 1.6730 - val_accuracy: 0.2437
Epoch 6/50
2/2 - 2s - loss: 1.7424 - accuracy: 0.2000 - val_loss: 1.6496 - val_accuracy: 0.1701
Epoch 7/50
2/2 - 2s - loss: 1.5676 - accuracy: 0.2000 - val_loss: 1.6359 - val_accuracy: 0.1701
Epoch 8/50
2/2 - 2s - loss: 1.5603 - accuracy: 0.4000 - val_loss: 1.6298 - val_accuracy: 0.1701
Epoch 9/50
2/2 - 2s - loss: 1.6607 - accuracy: 0.0000e+00 - val_loss: 1.6251 - val_accuracy: 0.1701
Epoch 10/50
2/2 - 2s - loss: 1.6053 - accuracy: 0.3000 - val_loss: 1.6223 - val_accuracy: 0.2437
Epoch 11/50
2/2 - 2s - loss: 1.